In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/GitHub/SSM")

include("/Users/ryansenne/Documents/GitHub/SSM//src/SSM.jl")
using .SSM
using LinearAlgebra
using Random
using Distributions
using Plots
using CSV
using DataFrames
using BenchmarkTools

  Activating project at `\Users\ryansenne\Documents\GitHub\SSM`


In [2]:
a = LDS()

LDS([0.41179258808564956 0.842098078747292; 0.30325630009931337 0.5512992155633235], [0.6119671119594139 0.44456574641721136; 0.21251974451682232 0.41771688590580125], nothing, Bool[1 0; 0 1], Bool[1 0; 0 1], [0.6099357661844094, 0.2980083297278924], [0.7040403535861083 0.4941860904739458; 0.11932455165956479 0.6734136861275365], nothing, 2, 2, Bool[1, 1, 1, 1, 1, 1, 1])

In [ ]:
# Generate data from a known dynamical system--a pendulum
g = 9.81 # gravity
l = 1.0 # length of pendulum
dt = 0.001 # time step
T = 0.004 # total time
# Discrete-time dynamics
A = [1.0 dt; -g/l*dt 1.0]
# Initial state
x0 = [0.0; 1.0]
# Time vector
t = 0:dt:T
# Generate data
x = zeros(2, length(t))
x[:,1] = x0
for i = 2:length(t)
    x[:,i] = A*x[:,i-1]
end

# Now lets add noise to the system and see if we can recover the dynamics
# Add driving Gaussian noise to simulate observations
x_noisy = zeros(2, length(t))
x_noisy[:, 1] = x0
observation_noise_std = 0.5

noise = rand(Normal(0, observation_noise_std), (2, length(t)))

for i in 2:length(t)
    x_noisy[:, i] = A * x[:, i-1] + noise[:, i]
end

# Define the LDS model parameters
H = I(2)  # Observation matrix (assuming direct observation)
Q = 1e-8 * I(2)  # Process noise covariance
R = 0.25 * I(2)  # Observation noise covariance
P0 = 1e-2 * I(2)  # Initial state covariance
x0 = [0.0; 1.0]  # Initial state mean

# Create the Kalman filter parameter vector
kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, ector([true, true, false, true, true, true, true, false]))

# Run the Kalman filter
x_filt, p_filt, x_pred, p_pred, v, F, K, ll = KalmanFilter(kf, Matrix{Float64}(x_noisy'))
# Run the smoother
x_smooth, P_smooth = KalmanSmoother(kf, Matrix{Float64}(x_noisy'))

In [9]:
# calculate the predictions of X
using Profile
x_prediction = zeros(length(t), 2)
x_prediction[1, :] = kf.x0
for t in 2:length(t)
    x_prediction[t, :] = kf.A * x_prediction[t-1, :]
end

@profile x_new = SSM.newton_raphson_tridg!(kf, x_prediction, x_noisy', 10, 1e-3)

UndefVarError: UndefVarError: `kf` not defined

In [13]:
using ForwardDiff
# H = ForwardDiff.hessian(x -> SSM.loglikelihood(x', kf, x_noisy[1:2, 1:100]'), x_prediction')
G = ForwardDiff.gradient(x -> SSM.loglikelihood(x', kf, x_noisy'), x_new')
G = reshape(G, 10, 1)

UndefVarError: UndefVarError: `x_new` not defined

In [2]:
# need to use previously aquired pendulum data for comparison
pendulum_data = CSV.read("/Users/ryansenne/Documents/GitHub/SSM/notebooks/pendulum_data_1.csv", DataFrame)

# convert x_noisy columns to matrix for use 
x_noisy =  Matrix(pendulum_data[1:10, 3:4])

# now create a Kalman Filter where we want to learn the parameters
A = [0.1 0.2; 0.3 0.4]
H = [1 0.0; 0.0 1]
Q = 1 * I(2)
R = 1 * I(2)
p0 = 1 * I(2)
x0 = [0.0, 1.0]

# kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, "Gaussian", Vector([true, true, false, true, true, true, true, false]))
kf_unknown = LDS(A, H, nothing, Q, R, x0, p0, nothing, 2, 2, [false, true, false, false, false, false])
# SSM.pca_init!(kf_unknown, x_noisy)

SSM.KalmanFilterEM!(kf_unknown, Matrix{Float64}(x_noisy), 100)

Marginal Log-likelihood at iteration 1: -27.713550116060578
Marginal Log-likelihood at iteration 2: -27.399564064097635
Marginal Log-likelihood at iteration 3: -27.275589744438086
Marginal Log-likelihood at iteration 4: -27.20863681213365
Marginal Log-likelihood at iteration 5: -27.16662624742429
Marginal Log-likelihood at iteration 6: -27.137783789697696
Marginal Log-likelihood at iteration 7: -27.11674851080136
Marginal Log-likelihood at iteration 8: -27.10072492432724
Marginal Log-likelihood at iteration 9: -27.088111031961734
Marginal Log-likelihood at iteration 10: -27.077922363260722
Marginal Log-likelihood at iteration 11: -27.069520397871873
Marginal Log-likelihood at iteration 12: -27.06247279683532
Marginal Log-likelihood at iteration 13: -27.056476360314416
Marginal Log-likelihood at iteration 14: -27.05131214201339
Marginal Log-likelihood at iteration 15: -27.046818074535015
Marginal Log-likelihood at iteration 16: -27.042871619833463
Marginal Log-likelihood at iteration 17

(LDS([0.1 0.2; 0.3 0.4], [1.0 0.0; 0.0 1.0], nothing, [1 0; 0 1], [1 0; 0 1], [0.0, 1.0], [0.009448495450195707 -0.0006339479777389345; -0.0006339479777389345 0.008993139891144475], nothing, 2, 2, Bool[0, 1, 0, 0, 0, 0]), -26.99197190480337)

In [5]:
x_filt, p_filt, x_pred, p_pred, v, S, K, ll, ml = SSM.KalmanFilter(kf_unknown, Matrix{Float64}(x_noisy))

([0.0 1.0; -0.458287172375825 0.5995594095642571; … ; -0.09094307393255335 0.594385340364292; -0.37264009014140687 0.38082636629703775], [0.5 0.0; 0.5058254459198622 0.012790400223250658; … ; 0.5062251504689818 0.013660099057141897; 0.506225150470281 0.013660099059975127;;; 0.0 0.5; 0.012790400223250687 0.5290807190530454; … ; 0.0136600990571419 0.5309733916813013; 0.013660099059975034 0.5309733916874813], [0.0 1.0; 0.2 0.4; … ; 0.07064245724862356 0.13603935235103218; 0.10978276067960308 0.21047121396595084], [1.0 0.0; 1.025 0.05500000000000001; … ; 1.0268475911281922 0.0590306357410916; 1.0268475911342276 0.059030635754287755;;; 0.0 1.0; 0.05500000000000001 1.125; … ; 0.0590306357410916 1.1337944299560774; 0.059030635754287755 1.1337944299849307], [0.0 0.0; -1.3117510767744314 0.4088926756141953; … ; -0.3427281781762949 0.8720355786768652; -0.9623063499821132 0.34559236916694647], [2.0 0.0; 2.025 0.05500000000000001; … ; 2.0268475911281922 0.0590306357410916; 2.026847591134228 0.0590

In [7]:
ml

-27.713550116060578

In [4]:
ll = SSM.loglikelihood(x_filt, kf_unknown, x_noisy)

-2.058166703974784

In [28]:
for i in 1:10
    z = ((kf_unknown.H * p_pred[i, :, :] * kf_unknown.H') + kf_unknown.R)
    println(det(z))
end

4.0
1.1077890013538432
0.9760529538224801
0.9398591439616736
0.9280005251778541
0.9239155790544029
0.9224841152765938
0.9219794969436195
0.9218012357723678
0.9217382166871365


In [21]:
p_filt[4, :, :]

2×2 Matrix{Float64}:
 -0.0555903  -0.072028
 -0.072028   -0.0128714

In [3]:
kf_unknown.R

2×2 Matrix{Float64}:
  0.300558  -0.170841
 -0.170841   0.387198